# ДЗ Text classification using CNN

## Задача (Sentiment Analysis)

Собраны твиты 2-ух тональностей, необходимо произвести классификацию на 2-а класса.

In [4]:
max_words = 2000
max_len = 40
num_classes = 1

# Training
epochs = 20
batch_size = 512
print_batch_n = 100

In [6]:
import pandas as pd

df_train = pd.read_csv('train.csv')
#df_test = pd.read_csv("/test.csv")
df_val = pd.read_csv("val.csv")

In [7]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [ ]:
df_test.head()

NameError: ignored

In [8]:
df_val.head()

,id,text,class
0,181467,RT @TukvaSociopat: Максимальный репост! ))) #є...,1
1,181468,чтоб у меня з.п. ежегодно индексировали на инд...,0
2,181469,@chilyandlime нехуя мне не хорошо !!! :((((,0
3,181470,"@inafish нее , когда ногами ахахах когда?ахаха...",0
4,181471,"Хочу сделать как лучше, а получаю как всегда. :(",0


### Предобработка

In [9]:
!pip install stop_words

  Created wheel for stop-words: filename=stop_words-2018.7.23-cp36-none-any.whl size=32917 sha256=ac7f2fa001baa2583ac07dcc59072d71b89d7e1a9efbdc513f8a088ca803b168
  Stored in directory: /root/.cache/pip/wheels/75/37/6a/2b295e03bd07290f0da95c3adb9a74ba95fbc333aa8b0c7c78
Successfully built stop-words


In [10]:
!pip install pymorphy2

     |████████████████████████████████| 51kB 2.9MB/s 
     |████████████████████████████████| 7.1MB 9.3MB/s 


In [13]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [14]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
#df_test['text'] = df_test['text'].apply(preprocess_text)

In [15]:
train_corpus = " ".join(df_train["text"])
train_corpus = train_corpus.lower()

In [16]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Отфильтруем данные

и соберём в корпус N наиболее частых токенов

In [17]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [52]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [53]:
tokens_filtered_top[:10]

['rt',
 'd',
 'знать',
 'ян',
 'любить',
 'завтра',
 'хороший',
 'делать',
 'новый',
 'смотреть']

In [54]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [55]:
import numpy as np
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [56]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["text"]], dtype=np.int32)
#x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["text"]], dtype=np.int32)
x_val = np.asarray([text_to_sequence(text, max_len) for text in df_val["text"]], dtype=np.int32)

In [57]:
x_train.shape

(181467, 40)

In [58]:
x_train[1]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   1, 202, 437, 158,
         6], dtype=int32)

# Keras model

In [66]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, MaxPool1D, Conv2D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  

In [60]:
df_val["class"].head(2)

0    1
1    0
Name: class, dtype: int64

In [91]:
num_classes = 2
y_train = keras.utils.to_categorical(df_train["class"], num_classes)
y_val = keras.utils.to_categorical(df_val["class"], num_classes)
#y_train = df_train['class'].values
#y_val = df_val['class'].values

In [92]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(20))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
#model.add(Dense(1, activation='sigmoid'))

In [93]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [94]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
319/319 [==============================] - 4s 14ms/step - loss: 0.5746 - accuracy: 0.6802 - val_loss: 0.5509 - val_accuracy: 0.6990
Epoch 2/20
319/319 [==============================] - 4s 13ms/step - loss: 0.5345 - accuracy: 0.7143 - val_loss: 0.5494 - val_accuracy: 0.7017
Epoch 3/20
319/319 [==============================] - 4s 12ms/step - loss: 0.5094 - accuracy: 0.7336 - val_loss: 0.5536 - val_accuracy: 0.7050


In [95]:
score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 0s 5ms/step - loss: 0.5569 - accuracy: 0.6992


Test score: 0.5568944811820984
Test accuracy: 0.6992461085319519


In [ ]:
results = model.predict(x_test, batch_size=batch_size, verbose=1)

45/45 [==============================] - 1s 15ms/step


Теперь посмотрим word2vec

In [22]:
import nltk
nltk.download('punkt')
df_train['text_token'] = df_train['text'].apply(lambda x: nltk.tokenize.word_tokenize(x))
df_train['text_token'].head(2)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


0    [alisachachkaн, уезжаааааааать, ❤, тожена, уез...
1    [rt, galyginvadim, ребята, девчата, кино, любо...
Name: text_token, dtype: object

In [23]:
import multiprocessing
import gensim
from gensim.models import Word2Vec

# Считываем файл с предобработанными твитами
from gensim.models import Word2Vec
modelW2V = Word2Vec(sentences=df_train['text_token'], size=4, window=5, min_count=3, sg = 1, hs = 0, negative = 10, workers=multiprocessing.cpu_count(), seed = 34)
# Обучаем модель 
#model = Word2Vec(data, size=200, window=5, min_count=3, workers=multiprocessing.cpu_count())

In [24]:
l = len(df_train['text_token'])
modelW2V.train([["hello", "world"]], total_examples=l, epochs=20)

(40, 40)

In [25]:
list(modelW2V.wv.vocab.keys())[:10]

['❤',
 'тожена',
 'уезжать',
 'rt',
 'galyginvadim',
 'ребята',
 'девчата',
 'кино',
 'любовь',
 'завтра']

In [26]:
x_train = df_train['text'].values
x_val = df_val['text'].values


In [27]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

def get_sequences(tokenizer, x):
    sequences = tokenizer.texts_to_sequences(x)
    return pad_sequences(sequences, maxlen=4)

# Cоздаем и обучаем токенизатор
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)

# Отображаем каждый текст в массив идентификаторов токенов
x_train_seq = get_sequences(tokenizer, x_train)
x_val_seq = get_sequences(tokenizer, x_val)

In [75]:
x_train_seq[0]

array([  0,   0, 565, 577], dtype=int32)

In [29]:
DIM = modelW2V.vector_size 
DIM

4

In [103]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=DIM, input_length=max_len))
size = 5
model.add(Conv1D(filters=200, kernel_size=DIM, activation='relu'))
model.add(GlobalMaxPool1D())
model.add(Dense(100))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [104]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [99]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss') 

history = model.fit(x_train_seq, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
319/319 [==============================] - 2s 5ms/step - loss: 0.5928 - accuracy: 0.6615 - val_loss: 0.5662 - val_accuracy: 0.6903
Epoch 2/20
319/319 [==============================] - 1s 4ms/step - loss: 0.5572 - accuracy: 0.6978 - val_loss: 0.5646 - val_accuracy: 0.6905
Epoch 3/20
319/319 [==============================] - 1s 4ms/step - loss: 0.5518 - accuracy: 0.7024 - val_loss: 0.5640 - val_accuracy: 0.6936
Epoch 4/20
319/319 [==============================] - 1s 4ms/step - loss: 0.5484 - accuracy: 0.7049 - val_loss: 0.5643 - val_accuracy: 0.6943


In [100]:
score = model.evaluate(x_val_seq, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 0s 2ms/step - loss: 0.5668 - accuracy: 0.6878


Test score: 0.5667983889579773
Test accuracy: 0.6878278851509094


In [ ]:
results = model.predict(x_test, batch_size=batch_size, verbose=1)

45/45 [==============================] - 1s 30ms/step
